In [1]:
#pip install mftool

In [2]:
#pip instal datetime

In [3]:
    #import pandas as pd
    #from mftool import Mftool
    #from datetime import datetime, timedelta

In [ ]:
mf = Mftool()

In [ ]:
# Getting the scheme codes of mutual funds
#scheme_codes = mf.get_scheme_codes()
#scheme_codes


In [ ]:
# Convert the scheme_codes dictionary to a DataFrame
#df = pd.DataFrame.from_dict(scheme_codes, orient='index', columns=['Scheme Name'])

# Save the DataFrame to a CSV file
#df.to_csv('scheme_codes.csv')

In [ ]:
    #Scheme Code: 101738
    # Date : 03-04-2006
    # current Date : 20-07-2023

In [1]:
import pandas as pd
from mftool import Mftool
from datetime import datetime, timedelta
import os

# Function to calculate returns for different time frames starting from a specific date
def calculate_returns1(latest_nav, nav_series, n):
    return (((latest_nav / nav_series.shift(n))) - 1)*100

# Function to calculate returns for different time frames starting from a specific date
def calculate_returns(latest_nav, nav_series, n):
    return (((latest_nav / nav_series.shift(n)) ** (1 / n)) - 1)*100

# Initialize Mftool
mf = Mftool()

# Manual input of scheme code and start date
scheme_code = input("Enter Scheme Code: ")
start_date = input("Enter start date (DD-MM-YYYY): ")

# Convert start_date to datetime object
start_date_obj = datetime.strptime(start_date, "%d-%m-%Y")

# Check if the input scheme code exists in the list of scheme codes
scheme_codes = mf.get_scheme_codes()
if scheme_code not in scheme_codes:
    print(f"Invalid Scheme Code '{scheme_code}'. Please enter a valid scheme code.")
else:
    # Create a list to store the results for the current scheme
    results = []

    # Fetch historical NAV data for the given scheme code and date range
    nav_data = mf.get_scheme_historical_nav(scheme_code, start_date_obj, datetime.today())

    # Extract the actual NAV data from the dictionary
    nav_data = nav_data['data']

    # Create a DataFrame from the NAV data
    nav_df = pd.DataFrame(nav_data)

    # Convert the "nav" column to numeric values
    nav_df["nav"] = pd.to_numeric(nav_df["nav"], errors="coerce")

    # Filter out any missing or zero NAV values
    nav_df = nav_df.dropna(subset=["nav"]).query("nav != 0")

    # Convert the "date" column to datetime objects
    nav_df["date"] = pd.to_datetime(nav_df["date"], format="%d-%m-%Y")

    # Sort the NAV data by date in ascending order
    nav_df.sort_values("date", inplace=True)

    # Calculate returns for different time frames starting from each date
    nav_df["1 Year Return"] = calculate_returns1(nav_df["nav"], nav_df["nav"], 249)
    nav_df["2 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250*2-1), 2)
    nav_df["3 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250*3-1), 3)
    nav_df["5 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250*5-1), 5)
    nav_df["10 Year Return"] = calculate_returns(nav_df["nav"], nav_df["nav"].shift(250*10-1), 10)


    # Filter the data to include only rows within the specified date range
    nav_df = nav_df[(nav_df["date"] >= start_date_obj) & (nav_df["date"] <= datetime.today())]

    # Retrieve the scheme name from the scheme_codes dictionary
    scheme_name = scheme_codes[scheme_code]

    # Add the scheme name and DataFrame to the results list
    results.append({"Scheme Code": scheme_code, "Scheme Name": scheme_name, "Data": nav_df})

    # Save the DataFrame to an Excel file in the same folder as the script
    output_file = os.path.join("E:/mftool/", f"{scheme_name}_{scheme_code}_returns.xlsx")
    nav_df.to_excel(output_file, index=False)

    print(f"Data for Scheme Code '{scheme_code}' and Scheme Name '{scheme_name}' saved to {output_file}")


Enter Scheme Code: 101738
Enter start date (DD-MM-YYYY): 03-04-2006
Data for Scheme Code '101738' and Scheme Name 'Aditya Birla Sun Life Dividend Yield Fund - Growth - Regular Plan' saved to E:/mftool/Aditya Birla Sun Life Dividend Yield Fund - Growth - Regular Plan_101738_returns.xlsx
